### `init-weights-1`: How to change the init of the layers
In this notebook, we just look at some basic getting/setting of nn layer weights.

In [12]:
import os, sys
import copy as copyroot
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt
from fastai.basics import *
from fastai.vision.all import *
import torch
from module2.mnist_helpers import build_df, seed_everything, build_dls
from module2.mnist_models import FeatsNet

In [ ]:
# %load_ext autoreload
# %autoreload 2
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = True

### Traditional Workflow

In [13]:
path = untar_data(URLs.MNIST_TINY)
df = build_df(path)

dls_tl = build_dls(target='topleft')

In [16]:
fn = FeatsNet(feats=['pts12', 'pts22'])
learn = Learner(dls_tl, 
                fn,
                loss_func=MSELossFlat(), 
                y_range=(-1.,1.),
                metrics=[R2Score()],             
               )

In [17]:
learn.fit(5,lr=2e-2)

epoch,train_loss,valid_loss,r2_score,time
0,0.077239,0.036989,0.273881,00:02
1,0.052865,0.026760,0.474685,00:02
2,0.041476,0.016357,0.678907,00:02
3,0.032718,0.015898,0.687912,00:02
4,0.027139,0.011512,0.774014,00:02


In [31]:
[param.shape for param in params]

[torch.Size([28, 64]), torch.Size([28]), torch.Size([2, 28])]

### Setting Weights Manually - OneLayer

In [73]:
model = FeatsNet(H=4, feats=['pts11'])

In [74]:
sd = model.linear1.state_dict()
sd2 = OrderedDict()
for k,v in sd.items():
    sd2[k] = v.zero_()

In [75]:
model.linear1.load_state_dict(sd2)

<All keys matched successfully>

### Setting Weights Manually - All Layers

In [76]:
model = FeatsNet(H=4, feats=['pts11'])

In [78]:
sd = model.state_dict()
for k in sd.keys():
    sd[k].zero_()

In [79]:
model.load_state_dict(sd)

<All keys matched successfully>

In [80]:
list(model.parameters())

[Parameter containing:
 tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]], requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0., 0.], requires_grad=True),
 Parameter containing:
 tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.]], requires_grad=True)]

In [92]:
def zero_p(nn_module, epsilon=1e-6):
    sd = nn_module.state_dict()
    for k in sd.keys(): 
        sd[k].zero_()
        sd[k] += epsilon
    nn_module.load_state_dict(sd)
    return nn_module

### Try fitting the zero init

In [94]:
fn = FeatsNet(feats=['pts12', 'pts22'])
fn = zero_p(fn)

In [95]:
# list(fn.parameters())

In [96]:
learn = Learner(dls_tl, 
                fn,
                loss_func=MSELossFlat(), 
                y_range=(-1.,1.),
                metrics=[R2Score()],             
               )

In [97]:
learn.fit(5,lr=2e-2)

epoch,train_loss,valid_loss,r2_score,time
0,0.176928,0.052999,-0.040393,00:03
1,0.115820,0.057562,-0.129976,00:03
2,0.088229,0.035979,0.293706,00:03
3,0.073125,0.033754,0.337396,00:03
4,0.062073,0.025912,0.491331,00:03
